In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data, wb
from yahoo_fin import stock_info as si
import datetime as dt
from fbprophet import Prophet
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf
import alpaca_trade_api as tradeapi
import time
import json


init_notebook_mode(connected=True)
cf.set_config_file(world_readable=True, offline=True)

In [ ]:
with open('keys.json', 'r') as f:
    keys = json.loads(f.read())

In [ ]:
def get_prophet_moves(ticker, daysOut=7, showCharts=False):
    # create start and end dates for stock data query
    start = dt.datetime.now() - dt.timedelta(days=365*5)
    end = dt.datetime.now()
    
    # get raw open, high, low, close, volume data
    try:
        stock_raw = data.DataReader(ticker, 'yahoo', dt.datetime.now() - dt.timedelta(days=365*10), dt.datetime.now())
    except:
        return 0, 0, 0
        
    # skip stocks with NaN in history
    if stock_raw.isnull().values.any():
        return 0, 0, 0
    
    # get the last close price
    last_close = stock_raw['Close'].values[-1]
        
    # convert datetimes
    stock_raw.reset_index(inplace=True)
    stock_df = stock_raw.copy()
    stock_df.loc[:, 'Date'] = pd.to_datetime(stock_raw.loc[:,'Date'], format="%Y-%m-%d")
        
    # prepare data for prophet
    ph_df = stock_df.drop(['Open', 'High', 'Low', 'Volume'], axis=1)
    ph_df.rename(columns={'Close': 'y', 'Date': 'ds'}, inplace=True)
    
    # fit data using prophet model
    m = Prophet()
    m.fit(ph_df)
    
    # create future dates
    future_prices = m.make_future_dataframe(periods=365)

    # predict prices
    forecast = m.predict(future_prices)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    
    # view results
    if showCharts:
        fig = m.plot(forecast)
        ax1 = fig.add_subplot(111)
        ax1.set_title(ticker + " Stock Price Forecast", fontsize=16)
        ax1.set_xlabel("Date", fontsize=12)
        ax1.set_ylabel("Close Price", fontsize=12)

        fig2 = m.plot_components(forecast)
        plt.show()

    # calculate predicted returns
    end_of_period = dt.datetime.now() + dt.timedelta(days=daysOut)
    
    future_close_max = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_upper
    future_close_expected = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat
    future_close_min = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_lower
    
    # calculate percent changes based on predictions
    max_move = (future_close_max - last_close)/last_close
    expected_move = (future_close_expected - last_close)/last_close
    min_move = (future_close_min - last_close)/last_close
        
    return (max_move, expected_move, min_move)

In [ ]:
def get_tickers_from_tradingview_copy():
    tickers = []
    with open("tickers_under20_marketcapdesc.txt") as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            tickers.append(lines[i].strip())
    return tickers

In [ ]:
def get_tickers_from_yahoo_copy():
    tickers = []
    with open("tickers_under20_yahoo.txt") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            tickers.append(lines[i].split()[0])
    return tickers

In [ ]:
def get_moves_for_tickers(tickers):
    moves = []
    for i, ticker in enumerate(tickers):
        try:
            max_move, expected_move, min_move = get_prophet_moves(ticker, 7)
            moves.append({
                'ticker': ticker,
                'max_move': max_move,
                'expected_move': expected_move,
                'min_move': min_move
            })
        except:
            time.sleep(15)
            max_move, expected_move, min_move = get_prophet_moves(ticker, 7)
            moves.append({
                'ticker': ticker,
                'max_move': max_move,
                'expected_move': expected_move,
                'min_move': min_move
            })
    return pd.DataFrame(moves)

In [ ]:
def run_strategy(chart=False, trade=False, paper=True):
    # get tickers
    tickers = get_tickers_from_tradingview_copy()
    
    # get moves from prophet
    moves = get_moves_for_tickers(tickers)
    
    if chart:
        trace1 = go.Bar(
            x=moves.ticker,
            y=moves.min_move,
            name='Minimum Move'
        )
        trace2 = go.Bar(
            x=moves.ticker,
            y=moves.expected_move,
            name='Expected Move'
        )
        trace3 = go.Bar(
            x=moves.ticker,
            y=moves.max_move,
            name='Maximum Move'
        )

        data = [trace1, trace2, trace3]
        layout = go.Layout(
            title='Stock Predictions',
            barmode='group'
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig)
    
    if paper:
        # init api
        api = tradeapi.REST(keys['alpaca_client_id'], keys['alpaca_client_secret'], 'https://paper-api.alpaca.markets')
        account = api.get_account()

        # get availible funds for trading
        min_buying_power = 75000
        avail_buying_power = float(account.buying_power) - min_buying_power

        if avail_buying_power < 0:
            raise Exception('No buying power availible')
        
        for ticker in tickers:
            # make sure ticker exists on Alpaca
            try:
                if not api.get_asset(ticker).tradable:
                    continue
            except:
                continue
                
            if moves[moves.ticker == ticker].expected_move.values[0] > 0:
                # get live quote price
                try:
                    last_quote = si.get_live_price(ticker)

                    # calculate amount of shares we can buy (round down using int division)
                    shares = (avail_buying_power/len(tickers))//last_quote

                    print(f'BUY: {ticker}, Shares: {shares}')
                    if trade:
                        api.submit_order(ticker, shares, 'buy', 'market', 'day')
                
                except Exception as e:
                    print(f"Error: Couldn't buy {ticker}: {e}")
                    continue
                        
            elif moves[moves.ticker == ticker].expected_move.values[0] < 0:
                # get current position
                try:
                    position = api.get_position(ticker)
                except Exception as e:
                    print(f"Error: Couldn't sell {ticker}: {e}")
                    continue
                    
                shares = int(position.qty)
                if shares > 2:
                    # sell half lmao
                    print(f'SELL: {ticker}, Shares: {shares//2}')
                    if trade:
                        api.submit_order(ticker, shares//2, 'sell', 'market', 'day')

In [ ]:
run_strategy(chart=True, trade=True)